In [78]:
import tensorflow as tf

from tensorflow import keras

import numpy as np

import pandas as pd

import os

import sys

import matplotlib.pyplot as plt

%matplotlib inline

#!{sys.executable} -m pip install opencv-python

import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras_metrics

from keras import optimizers



np.random.seed(123)

In [4]:
test_folder = '../data/test'
train_folder = '../data/train'
val_folder = '../data/val'

In [85]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64,64), batch_size = 627) 

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_folder, 
        target_size=(64,64), batch_size = 19)

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_folder, 
        target_size=(64,64), batch_size=5219)

Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [86]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [7]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 5216
Number of testing samples: 624
Number of validation samples: 16
train_images shape: (5216, 64, 64, 3)
train_labels shape: (5216, 2)
test_images shape: (624, 64, 64, 3)
test_labels shape: (624, 2)
val_images shape: (16, 64, 64, 3)
val_labels shape: (16, 2)


In [87]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(5216, 12288)
(624, 12288)
(16, 12288)


In [88]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))

# Modeling

## Baseline fully-connected

In [30]:
from keras import models
from keras import layers
import keras_metrics
np.random.seed(123)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
#model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
#model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers

model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [31]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_img, val_y),
                    verbose=1)

Train on 5216 samples, validate on 16 samples
Epoch 1/50
5216/5216 [==============================] - 1s 131us/step - loss: 0.5061 - acc: 0.7644 - val_loss: 0.9144 - val_acc: 0.5625
Epoch 2/50
5216/5216 [==============================] - 0s 82us/step - loss: 0.3625 - acc: 0.8522 - val_loss: 1.2546 - val_acc: 0.6250
Epoch 3/50
5216/5216 [==============================] - 0s 83us/step - loss: 0.2739 - acc: 0.8934 - val_loss: 0.5249 - val_acc: 0.7500
Epoch 4/50
5216/5216 [==============================] - 0s 83us/step - loss: 0.2222 - acc: 0.9118 - val_loss: 0.7708 - val_acc: 0.6875
Epoch 5/50
5216/5216 [==============================] - 0s 82us/step - loss: 0.2043 - acc: 0.9216 - val_loss: 0.4709 - val_acc: 0.8125
Epoch 6/50
5216/5216 [==============================] - 0s 83us/step - loss: 0.1880 - acc: 0.9258 - val_loss: 0.3695 - val_acc: 0.8750
Epoch 7/50
5216/5216 [==============================] - 0s 84us/step - loss: 0.1675 - acc: 0.9342 - val_loss: 0.6678 - val_acc: 0.7500
Epoch 8/

In [32]:
results_train = model.evaluate(train_img, train_y)

5216/5216 [==============================] - 0s 43us/step


In [33]:
results_test = model.evaluate(test_img, test_y)

624/624 [==============================] - 0s 42us/step


In [34]:
results_train

[0.07206388393222737, 0.9739263803680982]

In [35]:
results_test

[1.0638171571951647, 0.7451923076923077]

## CNN

In [89]:
model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [90]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=0.1, momentum=0.0, nesterov=False),
              metrics=[keras_metrics.recall()])

In [91]:
history = model.fit(train_images,
                    train_y,
                    epochs=2,
                    batch_size=32,
                    validation_data=(val_images, val_y),
                    verbose=1)

Train on 5216 samples, validate on 16 samples
Epoch 1/2
5216/5216 [==============================] - 16s 3ms/step - loss: 0.5739 - recall: 0.0239 - val_loss: 0.8173 - val_recall: 0.0000e+00
Epoch 2/2
5216/5216 [==============================] - 18s 3ms/step - loss: 0.5171 - recall: 0.0872 - val_loss: 1.1897 - val_recall: 0.1250


In [60]:
results_train = model.evaluate(train_images, train_y)

5216/5216 [==============================] - 24s 5ms/step


In [61]:
results_test = model.evaluate(test_images, test_y)

624/624 [==============================] - 3s 5ms/step


In [62]:
results_train

[0.5722823927373243, 0.0, 0.0]

In [63]:
results_test

[0.6865090620823395, 0.0, 0.0]

In [66]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu',
                        input_shape=(128,128,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [67]:
history = model.fit(train_images,
                    train_y,
                    epochs=1,
                    batch_size=32,
                    validation_data=(val_images, val_y),
                    verbose=1)

Train on 5216 samples, validate on 16 samples
Epoch 1/1
5216/5216 [==============================] - 44s 8ms/step - loss: 0.5778 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.7966 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


In [68]:
results_train = model.evaluate(train_images, train_y)

5216/5216 [==============================] - 15s 3ms/step


In [69]:
results_test = model.evaluate(test_images, test_y)

624/624 [==============================] - 2s 3ms/step


In [70]:
results_train

[0.5725939910104669, 0.0, 0.0]

In [71]:
results_test

[0.6894019811581342, 0.0, 0.0]

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 60, 60, 32)        8224      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 12544)             0         
__________